#**Задание**

Есть данные о цене битка от времени. Цену отрисовываем японскими свечами. Внизу барами
визуализируем объемы. На панели выводим выбор таймфрейма. Добавляем отрисовку
простейших индикаторов: RSI, Bollinger line, MA200. Настройки для каждого индикатора на панели
по клику.
Делаем без привлечения сторонних библиотек, пробуем через костыли в matplotlib и seaborn

---

**Выполнили:**
*Одинаев Георгий и Лопатин Никита*

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import TICKLEFT, TICKRIGHT, Line2D
from matplotlib.patches import Rectangle
import seaborn as sbn

from IPython.display import display, HTML, Markdown, clear_output
import ipywidgets as widgets
import json

import random
from datetime import datetime
import time

In [2]:
data = pd.read_csv('./bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')
data

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4857372,1617148560,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
4857373,1617148620,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
4857374,1617148680,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
4857375,1617148740,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [3]:
data = data.loc[data['Open'] > 0 ].reset_index(drop=True)
data

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
2,1325350740,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
3,1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
4,1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000
...,...,...,...,...,...,...,...,...
3613764,1617148560,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
3613765,1617148620,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
3613766,1617148680,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
3613767,1617148740,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [4]:
colors = ['red', 'blue', 'green', 'black', 'yellow', 'grey', 'lightblue', 'cyan', 'purple', 'orange']

In [5]:
def getDateTime(timestamp):
    return datetime.fromtimestamp(timestamp)

def getTime(timestamp):
    return datetime.fromtimestamp(timestamp)


In [6]:
# Time frame

date_start = widgets.DatePicker(
    description='Data start',
    value=getDateTime(data['Timestamp'].tail(100).min()),
    start=getDateTime(data['Timestamp'].min()),
    end=getDateTime(data['Timestamp'].max()),
  )
time_start = widgets.TimePicker(
    description='Time start',
    value=getDateTime(data['Timestamp'].tail(100).min()),
  )
date_finish = widgets.DatePicker(
    description='Data finish',
    value=getDateTime(data['Timestamp'].tail(100).max()),
    start=getDateTime(data['Timestamp'].min()),
    end=getDateTime(data['Timestamp'].max()),
  )
time_finish = widgets.TimePicker(
    description='Time finish',
    value=getDateTime(data['Timestamp'].tail(100).max()),
  )

start_box = widgets.HBox([date_start, time_start ])
finish_box = widgets.HBox([date_finish, time_finish ])
timeframe_box = widgets.VBox([start_box, finish_box ])

In [7]:
# Main
main_color_up = widgets.Dropdown(
    options=colors,
    value=colors[0],
    description='Color UP:'
  )

main_color_down = widgets.Dropdown(
    options=colors,
    value=colors[2],
    description='Color DOWN:'
  )

main_width = widgets.FloatSlider(
    value=0.5,
    min=0.1,
    max=1.0,
    step=0.1,
    description='Width:'
  )

main_box = widgets.VBox([main_color_up, main_color_down, main_width])

In [8]:
# RSI
RSI_toggle = widgets.Checkbox(
      description='Show RSI',
    )

RSI_color = widgets.Dropdown(
    options=colors,
    value=colors[0],
    description='RSI color:'
  )

RSI_line = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=2.0,
    step=0.1,
    description='RSI line:'
  )

RSI_box = widgets.VBox([RSI_toggle, RSI_color, RSI_line])

In [9]:
# Bollinger line
BL_toggle = widgets.Checkbox(
      description='Show BL',
    )

BL_color = widgets.Dropdown(
    options=colors,
    value=colors[0],
    description='BL color:'
  )

BL_line = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=2.0,
    step=0.1,
    description='BL line:'
  )

BL_box = widgets.VBox([BL_toggle, BL_color, BL_line])

In [10]:
# MA200
MA_toggle = widgets.Checkbox(
      description='Show MA200',
    )

MA_color = widgets.Dropdown(
    options=colors,
    value=colors[0],
    description='MA200 color:'
  )

MA_line = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=2.0,
    step=0.1,
    description='MA200 line:'
  )

MA_box = widgets.VBox([MA_toggle, MA_color, MA_line])

In [11]:
children = [timeframe_box, main_box, RSI_box, BL_box, MA_box]
tab = widgets.Tab()

tab.children = children
tab.set_title(0, 'Timeframe')
tab.set_title(1, 'Main')
tab.set_title(2, 'RSI')
tab.set_title(3, 'BL')
tab.set_title(4, 'MA200')

reload_btn = widgets.Button(description='Reload')

panel = widgets.HBox([tab, reload_btn])
graphic = widgets.Output()


In [92]:
def candlestick_ohlc(ax, quotes, width=0.2, colorup='k', colordown='r', alpha=1.0):
    k = 2000.0
    n = 1
    OFFSET = width / k

    lines = []
    patches = []
    
    step = 1
    if quotes.shape[0] > 500:
        step = quotes.shape[0] // 500 +1
        k = 12/step
        OFFSET = width / k
        
#     перебирает строки
    for q in range(0, quotes.shape[0], step):
        t, open, high, low, close= quotes.iloc[q][:5]
#             переобразовывает время в нужный формат
        
        if step != 1:
            count = 0
            for i in range(q, q+step):
                if i >= quotes.shape[0]:
                    break
                    
                t_i, open_i, high_i, low_i, close_i= quotes.iloc[i][:5]

                open+=open_i
                high+=high_i
                low+=low_i
                close+=close_i
                
                count+=1
            
            open/=count
            high/=count
            low/=count
            close/=count
            
    
        day_unixtime=time.strftime("%Y.%m.%d %H:%M", time.localtime(t))
        t=matplotlib.dates.date2num(datetime.strptime(day_unixtime,"%Y.%m.%d %H:%M"))
                
#        смотрит падает цена или нет от начальной цены дня
        if close >= open:
            color = colorup
            lower = open
            height = close - open
        else:
            color = colordown
            lower = close
            height = open - close
            
#        настройки палок для свечей
        vline = Line2D(
            xdata=(t, t), ydata=(low, high),
            color=color,
            linewidth=0.5,
            antialiased=True,
        )
    #        настройки тела для свечей
        rect = Rectangle(
            xy=(t - OFFSET, lower),
            width= width / (k/2),
            height=height,
            facecolor=color,
            edgecolor=color,
        )
        
        #        загрузака данных в график
        rect.set_alpha(alpha)
        
        lines.append(vline)
        patches.append(rect)
        
        
    for line in lines:
        ax.add_line(line)
        
    for rect in patches:
        ax.add_patch(rect)
    
    ax.autoscale_view()
    return lines, patches

In [93]:
def getTimestamp(date, time):
    return datetime(date.year, date.month, date.day, time.hour, time.minute, 0).timestamp()

In [94]:
def prepareData(data):
    global date_start
    global time_start
    global date_finish
    global time_finish
    
    ts_start = getTimestamp(date_start.value, time_start.value)
    ts_end = getTimestamp(date_finish.value, time_finish.value)
    
    data_end = data.loc[data['Timestamp'] <= ts_end]
    data_start_end = data_end.loc[data_end['Timestamp'] >= ts_start]
    
    
    return data_start_end

prepareData(data).shape

(100, 8)

In [95]:
def show_graphic(data):
    global main_color_up
    global main_color_down
    global main_width
    global RSI_toggle
    global RSI_color
    global RSI_line
    global BL_toggle
    global BL_color
    global BL_line
    global MA_toggle
    global MA_color
    global MA_line
    
    data_for_show = prepareData(data)
    
    # настройки графика
    fig,ax=plt.subplots(facecolor="white",figsize=(12,8))
    fig.subplots_adjust(bottom=0.1)
    ax.xaxis_date()
    plt.xticks(rotation=30)
    plt.title('Цена Битка')
    plt.xlabel('Дата')
    plt.ylabel('Цена')
    plt.grid(True)

    # отправляем функцию с данными для графика
    candlestick_ohlc(
        ax, 
        data_for_show, 
        width=main_width.value,
        colorup=main_color_up.value,
        colordown=main_color_down.value
    )

In [96]:
def on_click_reload(b):
    with graphic:
        clear_output(True)
        show_graphic(data)
        plt.show()
        

reload_btn.on_click(on_click_reload)

with graphic:
    clear_output(True)
    show_graphic(data)
    plt.show()

screen = widgets.VBox([graphic, panel])
screen